In [19]:
import pandas as pd 
import numpy as np

In [20]:
###show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [21]:
df=pd.read_excel('https://raw.githubusercontent.com/peterhi111/Econ-211/main/latest%20RRI.xlsx')
df.head()

,Internal RepRisk ID,Date of the month-end value. Time Slot Name.,Current RepRisk Index (RRI),RRI Trend,Peak RepRisk Index (RRI),Peak RepRisk Index (RRI) Date,RepRisk Rating,Country Sector Average,Environmental Percentage,Social Percentage,Governance Percentage,Legal Name of the Entity,ISIN
0,10,2020-08-31,21,-1,31,2020-04-04,BBB,38,0.0,1.0,0.0,Acer Inc,US0044341065
1,10,2020-08-31,21,-1,31,2020-04-04,BBB,38,0.0,1.0,0.0,Acer Inc,TW0002353000
2,10,2020-08-31,21,-1,31,2020-04-04,BBB,38,0.0,1.0,0.0,Acer Inc,US0044342055
3,100,2020-08-31,23,3,30,2020-02-28,A,17,0.5,0.5,0.0,Rio Tinto PLC,GB0007188757
4,100,2020-08-31,23,3,30,2020-02-28,A,17,0.5,0.5,0.0,Rio Tinto PLC,BRRIOTBDR007


In [22]:
df.describe()

,Internal RepRisk ID,Current RepRisk Index (RRI),RRI Trend,Peak RepRisk Index (RRI),Country Sector Average,Environmental Percentage,Social Percentage,Governance Percentage
count,2.673930e+05,267393.000000,267393.000000,267393.000000,267393.000000,267393.000000,267393.000000,267393.000000
mean,6.079697e+04,14.383163,-0.272827,26.845321,32.481262,0.111475,0.214815,0.329046
std,1.008151e+05,14.282821,3.942958,17.500596,14.013652,0.194795,0.284534,0.379871
min,2.000000e+00,0.000000,-13.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.537000e+03,0.000000,-1.000000,17.000000,23.000000,0.000000,0.000000,0.000000
50%,1.300500e+04,14.000000,-1.000000,27.000000,26.000000,0.000000,0.000000,0.170000
75%,8.666500e+04,24.000000,0.000000,37.000000,40.000000,0.180000,0.360000,0.640000
max,1.774029e+06,78.000000,46.000000,81.000000,100.000000,1.000000,1.000000,1.000000


As for the simplest case, we only look at the first company whose Internal RepRisk ID = 10

In [40]:
simu_data=df.loc[0].values
print(simu_data)

[10 Timestamp('2020-08-31 00:00:00') 21 -1 31
 Timestamp('2020-04-04 00:00:00') 'BBB' 38 0.0 1.0 0.0 'Acer Inc'
 'US0044341065']


Based on the methodology of RepRisk Rating (RRR), it is calculated by two factors - Peak RRI & Country-Sector Average. According to the simple data analysis we have done, we know that they have already provided the data of two factors. However, in the methodology RepRisk provided, we cannot how to generate the final rating with these two factors. Thus, I would first try to set 50% to each of the two factors seperately to see whether it works.

In [37]:
ave = (simu_data[4] + simu_data[7]) / 2
print(ave)

34.5


It's also weird that although RepRisk claimed that the RRR is a letter rating, they set the the data type of RRR as int. Thus, I made an assumption and tried to test the correctness of my assumption.

Since RepRisk regard RRR's data type as int, I assume each RRR level is corresponding to a certain region of scales (the whole region is from 0 - 100). 

In [38]:
simu_data2 = df.loc[3].values
simu_data2

array([100, Timestamp('2020-08-31 00:00:00'), 23, 3, 30,
       Timestamp('2020-02-28 00:00:00'), 'A', 17, 0.5, 0.5, 0.0,
       'Rio Tinto PLC', 'GB0007188757'], dtype=object)

In [52]:
ave2 = (simu_data2[4] + simu_data2[7]) / 2
print(ave2)

23.5


In [53]:
print(ave, simu_data[6])
print(ave2, simu_data2[6])

34.5 BBB
23.5 A


To make the assumption more reasonable, I also looked at another company's information and found that the one with lower average score of country-sector average and Peak RRI has a better rating. 

In [69]:
# Therefore, the following is how I let the letter rating correspond to 0 - 100 scales.
# char pred_rate
def rating(ave):
  pred_rate = ''
  if 0<=ave<=10:
    pred_rate = 'AAA'
  elif 10<ave<=20:
    pred_rate = 'AA'
  elif 20<ave<=30:
    pred_rate = 'A'
  elif 30<ave<=40:
    pred_rate = 'BBB'
  elif 40<ave<=50:
    pred_rate = 'BB'
  elif 50<ave<=60:
    pred_rate = 'B'
  elif 60<ave<=70:
    pred_rate = 'CCC'
  elif 70<ave<=80:
    pred_rate = 'CC'
  elif 80<ave<=90:
    pred_rate = 'C'
  else:
    pred_rate = 'D'
  print(pred_rate)

In [70]:
rating(ave)
print(simu_data[6])

BBB
BBB


In [71]:
rating(ave2)
print(simu_data2[6])

A
A


From the above, we can see that my assumption was verified and we can make a table for the corresponding regions of scales of 10 levels of letter grading. 

In [75]:
from prettytable import PrettyTable
x = PrettyTable(["RepRisk Rating(RRR)", "Scale"])
x.add_row(["AAA", "0-10"])
x.add_row(["AA", "11-20"])
x.add_row(["A", "21-30"])
x.add_row(["BBB", "31-40"])
x.add_row(["BB", "41-50"])
x.add_row(["B", "51-60"])
x.add_row(["CCC", "61-70"])
x.add_row(["CC", "71-80"])
x.add_row(["C", "81-90"])
x.add_row(["D", "91-100"])
print(x)

+---------------------+--------+
| RepRisk Rating(RRR) | Scale  |
+---------------------+--------+
|         AAA         |  0-10  |
|          AA         | 11-20  |
|          A          | 21-30  |
|         BBB         | 31-40  |
|          BB         | 41-50  |
|          B          | 51-60  |
|         CCC         | 61-70  |
|          CC         | 71-80  |
|          C          | 81-90  |
|          D          | 91-100 |
+---------------------+--------+
